In [1]:
import tensorflow as tf
from tensorflow.python.lib.io import file_io
from tqdm import tqdm
import numpy as np

2023-11-16 12:15:27.257436: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-16 12:15:32.116293: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /global/software/sl-7.x86_64/modules/langs/cuda/11.2/lib64:/global/home/groups/consultsw/sl-7.x86_64/modules/cudnn/7.0.5-cuda9.0/lib64:/global/software/sl-7.x86_64/modules/langs/python/3.7/lib
2023-11-16 12:15:32.116495: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open s

In [23]:
class Parser:
    def __init__(self, seq_length=524288, target_length=896, num_targets=5313, seq_1hot=True, seq_depth=4, seq_length_crop=None):
        self.seq_length = seq_length
        self.target_length = target_length
        self.num_targets = num_targets
        self.seq_1hot = seq_1hot
        self.seq_depth = seq_depth
        self.seq_length_crop = seq_length_crop

    def parse_proto(self, example_protos, raw=False):
        """Parse TFRecord protobuf."""

        # define features
        features = {
            TFR_INPUT: tf.io.FixedLenFeature([], tf.string),
            TFR_OUTPUT: tf.io.FixedLenFeature([], tf.string)
        }

        # parse example into features
        parsed_features = tf.io.parse_single_example(example_protos, features=features)

        # decode sequence
        sequence = tf.io.decode_raw(parsed_features[TFR_INPUT], tf.uint8)
#         if not raw:
#             if self.seq_1hot:
#                 sequence = tf.reshape(sequence, [self.seq_length])
#                 sequence = tf.one_hot(sequence, 1+self.seq_depth, dtype=tf.uint8)
#                 sequence = sequence[:,:-1]  # drop N
#             else:
#                 sequence = tf.reshape(sequence, [self.seq_length, self.seq_depth])
#             if self.seq_length_crop is not None:
#                 crop_len = (self.seq_length - self.seq_length_crop) // 2
#                 sequence = sequence[crop_len:-crop_len,:]
#             sequence = tf.cast(sequence, tf.uint8)

        # decode targets
        targets = tf.io.decode_raw(parsed_features[TFR_OUTPUT], tf.float16)
        if not raw:
            targets = tf.reshape(targets, [self.target_length, self.num_targets])
            targets = tf.cast(targets, tf.float32)

        return sequence, targets

In [24]:
TFR_INPUT = 'sequence'  # This needs to be the actual name used in your TFRecords for the sequence
TFR_OUTPUT = 'target'  # This needs to be the actual name used in your TFRecords for the targets

tfrs_path = '/clusterfs/nilah/richard/basenji2/data_pretrained_basenji2/human/tfrecords/'
tfr_filename = 'train-0-1.tfr'
tfr_output_filename = '/global/scratch/users/dixonluinenburg/cs282a_preprocessing/tfr_testing/recreate_train-0-1.tfr'
parser = Parser()
dataset = tf.data.TFRecordDataset(tfrs_path+tfr_filename,compression_type='ZLIB')
parsed_dataset = dataset.map(parser.parse_proto)    

In [40]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
def feature_bytes(values):
    """Convert numpy arrays to bytes features."""
    values = values.flatten().tobytes()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[values]))
def serialize_example(sequence, target):
    target = np.array(target).astype('float16')
    serialized_sequence = sequence.numpy().tobytes()
    
    features_dict = {
        TFR_INPUT: tf.train.Feature(bytes_list=tf.train.BytesList(value=[serialized_sequence])),
        TFR_OUTPUT: feature_bytes(target)
    }
    example = tf.train.Example(features=tf.train.Features(feature=features_dict))
    return example

In [37]:
with tf.io.TFRecordWriter(tfr_output_filename,options=options) as writer:
    for sequence,targets in tqdm(parsed_dataset,total=256):
        print(sequence,targets)

  1%|          | 3/256 [00:00<00:21, 11.79it/s]

tf.Tensor([0 0 0 ... 0 0 0], shape=(524288,), dtype=uint8) tf.Tensor(
[[0.07519531 0.01112366 0.01609802 ... 0.         0.         0.        ]
 [0.0881958  0.01392365 0.03829956 ... 0.         0.         0.        ]
 [0.05108643 0.01622009 0.01737976 ... 0.         0.         0.        ]
 ...
 [0.00142574 0.001544   0.         ... 0.         0.         0.        ]
 [0.00675964 0.         0.         ... 0.         0.         0.        ]
 [0.05609131 0.0559082  0.06762695 ... 0.         0.         0.        ]], shape=(896, 5313), dtype=float32)
tf.Tensor([0 0 0 ... 0 0 1], shape=(524288,), dtype=uint8) tf.Tensor(
[[0.06958008 0.06192017 0.         ... 0.14208984 0.         0.        ]
 [0.06665039 0.0425415  0.03186035 ... 0.         0.         0.19018555]
 [0.04772949 0.01244354 0.01963806 ... 0.         0.         0.        ]
 ...
 [0.09320068 0.06671143 0.         ... 0.         0.         0.        ]
 [0.09973145 0.03692627 0.01110077 ... 0.         0.         0.        ]
 [0.0589294

  3%|▎         | 7/256 [00:00<00:16, 14.95it/s]

tf.Tensor([0 0 1 ... 1 0 0], shape=(524288,), dtype=uint8) tf.Tensor(
[[0.08758545 0.1607666  0.10845947 ... 0.         0.         0.        ]
 [0.07965088 0.0758667  0.04248047 ... 0.         0.         0.        ]
 [0.06768799 0.05587769 0.05859375 ... 0.         0.         0.        ]
 ...
 [0.1071167  0.1348877  0.28125    ... 0.17297363 0.         0.        ]
 [0.08081055 0.0949707  0.18603516 ... 0.         0.         0.        ]
 [0.11560059 0.15039062 0.2631836  ... 0.         0.         0.        ]], shape=(896, 5313), dtype=float32)
tf.Tensor([0 1 0 ... 0 0 1], shape=(524288,), dtype=uint8) tf.Tensor(
[[0.0574646  0.06378174 0.01029968 ... 0.         0.         0.        ]
 [0.0047226  0.0048027  0.         ... 0.         0.         0.        ]
 [0.03335571 0.03765869 0.02542114 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.00190163 0.00617599 0.         ... 0.         0.         0.        ]
 [0.0082855

  4%|▍         | 11/256 [00:00<00:15, 15.79it/s]

...

 17%|█▋        | 43/256 [00:02<00:13, 15.89it/s]

tf.Tensor([0 1 0 ... 0 0 0], shape=(524288,), dtype=uint8) tf.Tensor(
[[0.04290771 0.05587769 0.10797119 ... 0.         0.         0.47729492]
 [0.02552795 0.0456543  0.02349854 ... 1.9658203  0.         0.        ]
 [0.05969238 0.03363037 0.02478027 ... 0.         0.         0.        ]
 ...
 [0.05557251 0.01374817 0.         ... 0.         0.         0.        ]
 [0.03494263 0.00950623 0.         ... 0.         0.         0.        ]
 [0.04495239 0.03408813 0.02253723 ... 0.         0.         0.        ]], shape=(896, 5313), dtype=float32)
tf.Tensor([0 0 0 ... 0 0 0], shape=(524288,), dtype=uint8) tf.Tensor(
[[0.06433105 0.040802   0.0859375  ... 0.         0.03089905 0.        ]
 [0.17578125 0.16601562 0.3005371  ... 0.         0.         0.        ]
 [0.1583252  0.3083496  0.06951904 ... 0.         0.03747559 0.        ]
 ...
 [0.09075928 0.05117798 0.05407715 ... 0.         0.         0.        ]
 [0.16003418 0.10217285 0.02896118 ... 0.0960083  0.         0.        ]
 [0.0866088

 18%|█▊        | 47/256 [00:02<00:13, 15.85it/s]

tf.Tensor([0 0 1 ... 0 0 1], shape=(524288,), dtype=uint8) tf.Tensor(
[[0.03799438 0.00718307 0.05953979 ... 0.         0.         0.        ]
 [0.03053284 0.04147339 0.0297699  ... 0.         0.17980957 0.        ]
 [0.04888916 0.04180908 0.04650879 ... 0.         0.         0.        ]
 ...
 [0.03016663 0.00961304 0.         ... 0.         0.         0.        ]
 [0.06665039 0.00791931 0.0791626  ... 0.         0.         0.        ]
 [0.05157471 0.01540375 0.01094055 ... 0.         0.         0.        ]], shape=(896, 5313), dtype=float32)
tf.Tensor([0 0 0 ... 1 0 0], shape=(524288,), dtype=uint8) tf.Tensor(
[[0.07208252 0.07507324 0.0218811  ... 0.         0.         0.        ]
 [0.04284668 0.04177856 0.05987549 ... 0.         0.         0.        ]
 [0.06896973 0.04998779 0.09460449 ... 0.         0.         0.        ]
 ...
 [0.07067871 0.04446411 0.00482941 ... 0.         0.         0.        ]
 [0.06079102 0.052948   0.03411865 ... 0.         0.         0.        ]
 [0.0341491

KeyboardInterrupt: 

In [52]:
options = tf.io.TFRecordOptions(compression_type="ZLIB")
# tfr_output_filename_2 = tfr_output_filename+'2'
with tf.io.TFRecordWriter(tfr_output_filename,options=options) as writer:
    for sequence,targets in tqdm(parsed_dataset,total=256):
#         print(targets.shape,type(targets))
        new_targets = tf.zeros((896,5313))
#         print(new_targets.shape,type(new_targets))
        example = serialize_example(sequence,new_targets)
#         print(example.SerializeToString()[0:100])
        writer.write(example.SerializeToString())

100%|██████████| 256/256 [00:36<00:00,  6.95it/s]


In [42]:
new_dataset = tf.data.TFRecordDataset(tfr_output_filename,compression_type='ZLIB')
new_parsed_dataset = new_dataset.map(parser.parse_proto)    

In [43]:
for sequence,targets in tqdm(new_parsed_dataset,total=256):
    print(sequence,targets)

  1%|          | 3/256 [00:00<00:21, 12.02it/s]

tf.Tensor([0 0 0 ... 0 0 0], shape=(524288,), dtype=uint8) tf.Tensor(
[[0.07519531 0.01112366 0.01609802 ... 0.         0.         0.        ]
 [0.0881958  0.01392365 0.03829956 ... 0.         0.         0.        ]
 [0.05108643 0.01622009 0.01737976 ... 0.         0.         0.        ]
 ...
 [0.00142574 0.001544   0.         ... 0.         0.         0.        ]
 [0.00675964 0.         0.         ... 0.         0.         0.        ]
 [0.05609131 0.0559082  0.06762695 ... 0.         0.         0.        ]], shape=(896, 5313), dtype=float32)
tf.Tensor([0 0 0 ... 0 0 1], shape=(524288,), dtype=uint8) tf.Tensor(
[[0.06958008 0.06192017 0.         ... 0.14208984 0.         0.        ]
 [0.06665039 0.0425415  0.03186035 ... 0.         0.         0.19018555]
 [0.04772949 0.01244354 0.01963806 ... 0.         0.         0.        ]
 ...
 [0.09320068 0.06671143 0.         ... 0.         0.         0.        ]
 [0.09973145 0.03692627 0.01110077 ... 0.         0.         0.        ]
 [0.0589294

  3%|▎         | 7/256 [00:00<00:15, 15.96it/s]

tf.Tensor([0 0 1 ... 1 0 0], shape=(524288,), dtype=uint8) tf.Tensor(
[[0.08758545 0.1607666  0.10845947 ... 0.         0.         0.        ]
 [0.07965088 0.0758667  0.04248047 ... 0.         0.         0.        ]
 [0.06768799 0.05587769 0.05859375 ... 0.         0.         0.        ]
 ...
 [0.1071167  0.1348877  0.28125    ... 0.17297363 0.         0.        ]
 [0.08081055 0.0949707  0.18603516 ... 0.         0.         0.        ]
 [0.11560059 0.15039062 0.2631836  ... 0.         0.         0.        ]], shape=(896, 5313), dtype=float32)
tf.Tensor([0 1 0 ... 0 0 1], shape=(524288,), dtype=uint8) tf.Tensor(
[[0.0574646  0.06378174 0.01029968 ... 0.         0.         0.        ]
 [0.0047226  0.0048027  0.         ... 0.         0.         0.        ]
 [0.03335571 0.03765869 0.02542114 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.00190163 0.00617599 0.         ... 0.         0.         0.        ]
 [0.0082855

  4%|▍         | 11/256 [00:00<00:14, 16.55it/s]

...

tf.Tensor([1 0 0 ... 1 0 0], shape=(524288,), dtype=uint8) tf.Tensor(
[[0.0198822  0.0058136  0.07501221 ... 0.         0.         0.        ]
 [0.07073975 0.05334473 0.08239746 ... 0.         0.         0.        ]
 [0.04174805 0.04296875 0.02268982 ... 0.         0.         0.        ]
 ...
 [0.10803223 0.08044434 0.04553223 ... 0.         0.9838867  0.        ]
 [0.09527588 0.03405762 0.17248535 ... 0.7470703  0.984375   0.        ]
 [0.10955811 0.09741211 0.08563232 ... 0.2364502  0.8496094  0.        ]], shape=(896, 5313), dtype=float32)
tf.Tensor([0 0 1 ... 1 0 0], shape=(524288,), dtype=uint8) tf.Tensor(
[[0.09991455 0.06030273 0.02365112 ... 0.         0.         0.        ]
 [0.1217041  0.04138184 0.04086304 ... 0.         0.         0.26513672]
 [0.09783936 0.08081055 0.07177734 ... 0.         0.14221191 0.04666138]
 ...
 [0.03936768 0.04989624 0.05712891 ... 0.         0.         0.        ]
 [0.11383057 0.14086914 0.08306885 ... 0.         0.         0.        ]
 [0.0162811

 92%|█████████▏| 235/256 [00:14<00:01, 16.64it/s]

tf.Tensor([0 0 0 ... 0 0 0], shape=(524288,), dtype=uint8) tf.Tensor(
[[0.04931641 0.05651855 0.00965881 ... 0.         0.         0.        ]
 [0.03424072 0.02105713 0.01287842 ... 0.         0.         0.        ]
 [0.05230713 0.03521729 0.07592773 ... 0.         0.         0.        ]
 ...
 [0.08496094 0.06060791 0.22399902 ... 0.         0.         0.        ]
 [0.01812744 0.02580261 0.06115723 ... 0.         0.         0.        ]
 [0.02302551 0.00608063 0.         ... 0.         0.         0.        ]], shape=(896, 5313), dtype=float32)


 92%|█████████▏| 235/256 [00:14<00:01, 15.88it/s]


KeyboardInterrupt: 

In [51]:
for (sequence1,targets1), (sequence2,targets2) in tqdm(zip(parsed_dataset,new_parsed_dataset),total=256):
    if not tf.reduce_all((tf.equal(sequence1,sequence2))):
        print('Sequences are not identical')
#     else:
#         print('Sequences are identical')
    if not tf.reduce_all(tf.equal(targets1,targets2)):
        print('Targets are not identical')
#     else:
#         print('Targets are identical')

  0%|          | 0/256 [00:00<?, ?it/s]


InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Input to reshape is a tensor with 8960 values, but the requested shape has 4760448
	 [[{{node Reshape}}]] [Op:IteratorGetNext]